In [38]:
import pandas as pd
import jieba
from gensim import corpora, models, similarities

In [28]:
data = pd.read_csv('./data/result.csv',encoding="utf8",error_bad_lines=False, sep=None)

/Users/exudingtao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [29]:
data = data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [30]:
data = data[['微博','老师or学生']]

In [31]:
data

,微博,老师or学生
126,#延假期不停学##停课不停学#大家有问题快来咨询，老师在线帮你解答~,tea
128,#延假期不停学##停课不停学#闷在家的正确姿势#教育在行动#,tea
129,#延假期不停学##停课不停学#假期不停学,tea
130,#延假期不停学##停课不停学#假期不停学,tea
136,//@头条教育:#延假期不停学##停课不停学#大家有问题快来咨询，老师在线帮你解答~,tea
...,...,...
385172,#朗阁温馨提示#请细细看这9张图❗️现在不出门非常关键❗️居家隔离❗️好好做“D”❗️继续做...,tea
385936,学习了//@刘码露:实用//@江南大野花:存了了解分类//@沐非://@管鑫Sa:协和医生写...,leaner
385947,转发学习//@shly_yyy:转首页//@眼儿媚滴春://@沐非://@管鑫Sa:协和医生...,tea
385951,学习//@刘码露:实用//@江南大野花:存了了解分类//@沐非://@管鑫Sa:协和医生写的...,leaner


In [32]:
content = data['微博'].values.tolist()

In [33]:
#去停用词
stop_word_path = './data/stopword.txt'
stop_word_list = [sw.replace('\n','') for sw in open(stop_word_path).readlines()]

def preprocess_text(content_lines,sentences):
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = list(filter(lambda x: x not in stop_word_list,segs))
            sentences.append(' '.join(segs))#分完词后 用空格串起来 且和label合并为一个元组
        except Exception as e:
            print(line)
            continue

In [34]:
#生成训练数据
sentences = []
preprocess_text(content,sentences)

In [42]:
train = []
for line in sentences:
    temp = []
    line_list = line.split(' ')
    for item in line_list:
        temp.append(item)
    train.append(temp)

In [47]:
#构建词频矩阵，训练LDA模型
"""构建词频矩阵，训练LDA模型"""
dictionary = corpora.Dictionary(train) #构建词典
## corpus是把每条新闻ID化后的结果，每个元素是新闻中的每个词语，在字典中的ID和频率
corpus = [dictionary.doc2bow(text) for text in train] #文本id化

In [50]:
lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)

In [52]:
topic_list = lda.print_topics(10)

In [54]:
print("10个主题的单词分布为：\n")
for topic in topic_list:
    print(topic)

10个主题的单词分布为：

(0, '0.011*"🙏" + 0.010*"_" + 0.009*"网上" + 0.008*"学习" + 0.008*"找" + 0.008*"希望" + 0.008*"没有" + 0.007*"资源" + 0.007*"不" + 0.007*"…"')
(1, '0.020*"都" + 0.018*"人" + 0.011*"不要" + 0.009*"学习" + 0.009*"在家" + 0.009*"视频" + 0.008*"不" + 0.007*"_" + 0.006*"找" + 0.006*"冻毙"')
(2, '0.063*"转发" + 0.018*"##" + 0.017*"都" + 0.015*"被删" + 0.015*"山西" + 0.014*"专业" + 0.012*"少女" + 0.012*"记得" + 0.010*"疫情" + 0.010*"已经"')
(3, '0.012*"被删" + 0.012*"转发" + 0.011*"🙏" + 0.010*"_" + 0.009*"少女" + 0.008*"视频" + 0.008*"学习" + 0.008*"一次" + 0.008*"资源" + 0.008*"看到"')
(4, '0.014*"转发" + 0.006*"被删" + 0.006*"…" + 0.006*"不" + 0.006*"都" + 0.006*"不要" + 0.006*"转" + 0.006*"大家" + 0.005*"没有" + 0.004*"老虎机"')
(5, '0.018*"公务员" + 0.011*"##" + 0.010*"学习" + 0.009*"疫情" + 0.009*"说" + 0.009*"没有" + 0.009*"山西" + 0.009*"专业" + 0.008*"资源" + 0.008*"网上"')
(6, '0.012*"🙏" + 0.011*"_" + 0.006*"转" + 0.006*"学习" + 0.005*"小" + 0.005*"太" + 0.005*"记者" + 0.005*"转发" + 0.005*"证据" + 0.005*"新生代"')
(7, '0.028*"🙏" + 0.014*"_" + 0.011*"好" + 0.010*"去世" + 0.009*"

In [56]:
"""第三步：查看某封邮件所属的主题"""
print("第1封邮件的大致内容为：\n",train[0],'\n')
topic = lda.get_document_topics(corpus[0])
print("第1封邮件的主题分布为：\n",topic,'\n')

第1封邮件的大致内容为：
 ['延', '假期', '不停', '学', '##', '停课', '不停', '学', '大家', '问题', '快', '咨询', '老师', '在线', '帮', '解答', '~'] 

第1封邮件的主题分布为：
 [(8, 0.9499897)] 



In [170]:
!pip install pyspark

     |██▍                             | 16.6MB 26kB/s eta 2:06:289     |█▎                              | 8.6MB 30kB/s eta 1:53:39ERROR: Exception:
Traceback (most recent call last):
  File "/Users/exudingtao/opt/anaconda3/lib/python3.7/site-packages/pip/_vendor/urllib3/contrib/pyopenssl.py", line 304, in recv_into
    return self.connection.recv_into(*args, **kwargs)
  File "/Users/exudingtao/opt/anaconda3/lib/python3.7/site-packages/OpenSSL/SSL.py", line 1822, in recv_into
    self._raise_ssl_error(self._ssl, result)
  File "/Users/exudingtao/opt/anaconda3/lib/python3.7/site-packages/OpenSSL/SSL.py", line 1622, in _raise_ssl_error
    raise WantReadError()
OpenSSL.SSL.WantReadError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/exudingtao/opt/anaconda3/lib/python3.7/site-packages/pip/_vendor/urllib3/contrib/pyopenssl.py", line 304, in recv_into
    return self.connection.recv_into(*args, **kwargs)
  File "/Users